<a href="https://colab.research.google.com/github/Prajna1999/eng-or-mt/blob/master/tokenizer_preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install --upgrade transformers sentencepiece  huggingface_hub datasets


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 69.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 481.2/481.2 kB 28.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 26.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 12.3 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.0
    Uninstalling fsspec-2025.3.0:
      Successfully uninstalled fsspec-2025.3.0
  Attempting uninstall: huggingface_hub
    Found existing installation: huggingface-hub 0.29.3
    Uninstalling huggingface-hub-0.29.3:
      Successfully uninstalled huggingface-hub-0.29.3
  Attempting uninstall: transformers
    Found existing installation: transformers 4.

In [3]:
from huggingface_hub import login
login()

In [4]:
from transformers import AutoTokenizer
base_mistral_tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.3", use_fast=False)

tokenizer_config.json:   0%|          | 0.00/141k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/587k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

In [18]:
import unicodedata
import regex
class OdiaTokenizerWrapper:
    def __init__(
        self,
        base_tokenizer,
        suffixes=None,
        named_entities=None,
        compound_roots=None,
        force_suffix_split=False,
        enable_fuzzy_split=False
        ):
      self.tokenizer=base_tokenizer
      self.vocab=self.tokenizer.get_vocab()

      # known Odia suffixes. In future expand the list
      self.suffixes=suffixes or[ "ମାନେ", "ଗୁଡ଼ିକ", "ରେ", "କୁ", "ରୁ", "ଟି", "ଙ୍କ","ଙ୍କୁ"]
      self.named_entities=named_entities or ["ଜଗନ୍ନାଥ", "ଭୁବନେଶ୍ୱର", "କଟକ", "ପୁରୀ", "ବାଲେଶ୍ୱର", "ସମ୍ବଲପୁର",
    "ଓଡ଼ିଶା", "ଝାରସୁଗୁଡ଼ା", "ଗଞ୍ଜାମ", "ମୟୂରଭଞ୍ଜ", "ନୟାଗଡ଼", "କଳାହାଣ୍ଡି",
    "ଖୋର୍ଦ୍ଧା", "ନୂଆପଡା", "ନବରଙ୍ଗପୁର", "ସୁନ୍ଦରଗଡ଼", "ବରଗଡ଼", "କନ୍ଧମାଳ",
    "ଗଞ୍ଜେଶ୍ୱର", "ଗୋପାଳପୁର", "ପାରାଦୀପ", "ଅନୁଗୁଳ", "ଦେବଗଡ଼", "କେନ୍ଦୁଝର",
    "ବୌଦ୍ଧ", "ମଲକାନଗିରି", "ଗଣେଶ", "ଶିବ", "ଲିଙ୍ଗରାଜ", "ମାଓ", "ମହାନଦୀ",
    "ଚିଲିକା", "ରଥଯାତ୍ରା", "କୋଣାର୍କ", "ସୁର୍ଯ୍ୟମନ୍ଦିର", "ଧୂଳିଗିରି", "ନନ୍ଦନକାନନ",
    "ଗଣେଶପୂଜା", "ଦୁର୍ଗାପୂଜା", "କାଳୀପୂଜା", "ଲକ୍ଷ୍ମୀପୂଜା", "ସରସ୍ୱତୀପୂଜା",
    "ପ୍ରଭାତୀ", "ମହାପ୍ରଭୁ", "ମାଁତରିଣୀ", "ମାଁସାମଲେଶ୍ୱରୀ", "ମାଁମଣିକେଶ୍ୱରୀ",
    "ମାଁକାନକଦୁର୍ଗା", "ବିମଳା", "ଭାଗବତ", "ଗୀତା", "ରାମାୟଣ", "ମହାଭାରତ",
    "ଗୋପବନ୍ଧୁ", "ବିଜୁପଟ୍ନାୟକ", "ନବୀନପଟ୍ନାୟକ", "ହରିହର", "ବିରାଜା", "ଚନ୍ଦ୍ରଭାଗା",
    "ମହେନ୍ଦ୍ରତନୟ", "ଅଶୋକ", "ଧରିତ୍ରୀ", "ସାମ୍ବ", "ନୀଳାଚଳ", "ପଟିଆ", "ନୟାପଲ୍ଲୀ"]
      self.compound_roots=compound_roots or [

                                            "ଶିକ୍ଷା", "ନୀତି", "ସଂସ୍କୃତି", "କେନ୍ଦ୍ର", "ବିଭାଗ", "ପୁସ୍ତକ", "ମେଳା",
    "ପ୍ରଶାସନ", "ନିୟମ", "ପରୀକ୍ଷା", "ମନ୍ତ୍ରଣାଳୟ", "ପ୍ରଧାନ", "ମନ୍ତ୍ରୀ",
    "ବିଶ୍ୱବିଦ୍ୟାଳୟ", "ସଂସ୍ଥା", "ନିୟୋଜନ", "ବିଭାଗ", "ସମିତି", "ଅଧିକାରୀ",
    "ଚିକିତ୍ସା", "ପ୍ରତିଷ୍ଠାନ", "ଶିଶୁ", "ସେବା", "ଉଦ୍ୟୋଗ", "ଆୟୋଜନ",
    "ତଥ୍ୟ", "ପ୍ରଯୁକ୍ତି", "ଖେଳ", "କ୍ରୀଡା", "ଅଭିଯାନ", "ଅନୁଷ୍ଠାନ",
    "ସାଂସ୍କୃତିକ", "ବିଜ୍ଞାନ", "ତନ୍ତ୍ର", "ଅନୁସନ୍ଧାନ", "ପ୍ରକାଶନ",
    "ବ୍ୟବସ୍ଥା", "ବଜେଟ", "ଉଦ୍‌ଘାଟନ", "ଅଧିବେଶନ", "ବୈଠକ", "ଉପସ୍ଥାପନ",
    "ନୀତି", "ଯୋଜନା", "ପଦକ୍ଷେପ", "ପ୍ରତିବେଦନ", "ସମୀକ୍ଷା", "ସୂଚନା",
    "ସୁରକ୍ଷା", "ନିର୍ବାଚନ", "ପ୍ରଚାର", "ନିୟମାବଳୀ", "ନିୟମକ", "ସଚିବ",
    "ପରିଚାଳନା", "ପାଠ୍ୟକ୍ରମ", "ଅନୁମୋଦନ", "ଶ୍ରମ", "ଉନ୍ନତି", "ଅନୁଶାସନ"
      ]
      self.force_suffix_split=force_suffix_split
      self.enable_fuzzy_split=enable_fuzzy_split

    def normalize_unicode(self, text):
      normalized_text=unicodedata.normalize("NFC", text)
      cleaned = normalized_text.replace('\u200d', '').replace('\u200c', '')
      return cleaned


    def is_known_token(self,token):
      # sentencepiece tokenizer sometimes have "__token" structure in their vocab
      return token in self.vocab or f"__{token}" in self.vocab

    def get_graphemes(self,word):
      return regex.findall(r'\X', word)

    def is_named_entity(self, word):
      for entity in self.named_entities:
          if self.enable_fuzzy_split:
              if word.startswith(entity) or word == entity:
                  return True
          else:
              if word == entity:
                  return True
      return False

    def compound_splitter(self,word):
      graphemes=self.get_graphemes(word)
      for i in range(2,len(graphemes)):
        left="".join(graphemes[:i])
        right="".join(graphemes[i:])

        if left in self.compound_roots and right in self.compound_roots:
          # print(f"Compound root split: '{word}' → '{left}' + '{right}'")
          return f"{left} {right}"
      return word

    def split_hyphenated(self, word):
      if "-" in word:
          parts = word.split("-")
          if len(parts) == 2 and all(len(p) > 1 for p in parts):
              # print(f"🔨 Hyphen split: '{word}' → '{parts[0]} {parts[1]}'")
              return f"{parts[0]} {parts[1]}"
      return word

    def split_suffix(self, word, depth=0):
        indent = "  " * depth
        graphemes = self.get_graphemes(word)

        for i in range(1, len(graphemes)):
            stem = "".join(graphemes[:i])
            suffix = "".join(graphemes[i:])

            if suffix in self.suffixes and len(stem) > 1:
              # disable unknown token check on an ad hoc basis
                if self.force_suffix_split or  not self.is_known_token(stem) or not self.is_known_token(suffix):

                    # print(f"{indent}📍 Split (grapheme-safe): '{word}' → '{stem}' + '{suffix}'")
                    split_stem = self.split_suffix(stem, depth + 1)
                    return f"{split_stem} {suffix}"
                else:
                    print(f"{indent}⚠️ Not splitting: '{word}' → parts unknown")
        print(f"{indent}✅ Final stem: '{word}'")
        return word


    def preprocess_text(self,text):
      normalized=self.normalize_unicode(text)
      words=normalized.split()

      processed=[]
      for word in words:

        if self.is_named_entity(word):
            print(f"Skipping named entity(fuzzy): {word}")
            processed.append(word)
        else:
            word = self.split_hyphenated(word)
            word = self.compound_splitter(word)
            word = self.split_suffix(word)
            processed.append(word)
      return " ".join(processed)

    def visualize_token_split(self, text, return_stats=False):
      print("\n🧪 ORIGINAL INPUT")
      print("📌 Text:", text)

      # --- Raw tokenization ---
      raw_tokens = self.tokenizer.tokenize(text)
      print("\n🧩 RAW TOKENIZATION")
      print(f"🧱 Tokens ({len(raw_tokens)}):", raw_tokens)

      # --- Preprocessing ---
      split_text = self.preprocess_text(text)

      # --- Preprocessed tokenization ---
      pre_tokens = self.tokenizer.tokenize(split_text)
      print("\n🛠️ AFTER PREPROCESSING")
      print("🧼 Cleaned:", split_text)
      print(f"🔤 Tokens ({len(pre_tokens)}):", pre_tokens)

      # --- Comparison ---
      print("\n📊 TOKEN COUNT COMPARISON")
      delta = len(raw_tokens) - len(pre_tokens)
      if delta > 0:
          print(f"✅ Token count reduced by: {delta} tokens")
      elif delta < 0:
          print(f"⚠️ Token count increased by: {-delta} tokens")
      else:
          print("➖ Token count unchanged")

      print("—" * 50)

      if return_stats:
          return {
              "original": text,
              "preprocessed": split_text,
              "raw_tokens": raw_tokens,
              "pre_tokens": pre_tokens,
              "delta": delta
          }

    def encode(self, text, **kwargs):
      preprocessed=self.preprocess_text(text)
      encoded=self.tokenizer.encode(preprocessed, **kwargs)
      return encoded


In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [22]:
# clean the dataframe fields for odia and english
import re
def clean_english(text):
  if not isinstance(text,str) or pd.isna(text):
    return ""

  text=unicodedata.normalize('NFKC', text)

  text=re.sub(r'\s+',' ',  text)
      # Remove URLs
  text = re.sub(r'https?://\S+|www\.\S+', '', text)

  # Remove HTML tags
  text = re.sub(r'<.*?>', '', text)

  # Standardize punctuation spacing
  text = re.sub(r'([.,!?()])', r' \1 ', text)
  text = re.sub(r'\s+', ' ', text).strip()

  return text

In [23]:
odia_tokenizer=OdiaTokenizerWrapper(base_tokenizer=base_mistral_tokenizer)
def preprocess_dataframe(df):
  df_clean=df.copy()

  df_clean['eng_string_clean']=df_clean['eng_string'].apply(clean_english)
  df_clean['or_string_clean']=df_clean['or_string'].apply(odia_tokenizer.preprocess_text)

  df_clean=df_clean.reset_index(drop=True)
  return df_clean

In [ ]:
df=pd.read_csv('/content/drive/My Drive/bpcc_eng_odia.csv')
cleaned_df=preprocess_dataframe(df)

In [25]:
import pandas as pd
from datasets import Dataset, DatasetDict
from sklearn.model_selection import train_test_split

def convert_to_huggingface_dataset(df, test_size=0.1, val_size=0.1, random_state=42):
  df_copy=df.copy()

  train_df,test_df=train_test_split(df_copy, test_size=test_size, random_state=random_state)
  train_df,val_df=train_test_split(train_df, test_size=test_size, random_state=random_state)

  dataset_dict=DatasetDict({
      "train":Dataset.from_pandas(train_df),
      "validation":Dataset.from_pandas(val_df),
      "test":Dataset.from_pandas(test_df)
  })

  return dataset_dict


In [26]:
hf_dataset=convert_to_huggingface_dataset(cleaned_df)

In [30]:
hf_dataset

DatasetDict({
    train: Dataset({
        features: ['eng_string', 'or_string', 'eng_string_clean', 'or_string_clean', '__index_level_0__'],
        num_rows: 73256
    })
    validation: Dataset({
        features: ['eng_string', 'or_string', 'eng_string_clean', 'or_string_clean', '__index_level_0__'],
        num_rows: 8140
    })
    test: Dataset({
        features: ['eng_string', 'or_string', 'eng_string_clean', 'or_string_clean', '__index_level_0__'],
        num_rows: 9044
    })
})

In [91]:
# format for finetuning translation
import json
def format_for_translation(example, template_format="chatml"):
  eng_text=example['eng_string_clean']
  or_text=example['or_string_clean']
  if template_format=="chatml":
    formatted={
         "messages": [
                {"role": "system", "content": "You are a helpful translation assistant that translates English to Odia."},
                {"role": "user", "content": f"Translate the following English text to Odia: {eng_text}"},
                {"role": "assistant", "content": or_text}
            ]
    }
  elif template_format=="alpaca":
        # Alpaca format
        formatted = {
            "instruction": "Translate the following English text to Odia.",
            "input": eng_text,
            "output": or_text
        }
  elif template_format=="llama2":
    formatted={
        "text": f"<s>[INST] <<SYS>>\nYou are a helpful translation assistant that translates English to Odia.\n<</SYS>>\n\nTranslate the following English text to Odia: {eng_text} [/INST] {or_text}</s>"
    }
  elif template_format=="text_only":
    formatted={
         "text": f"English: {eng_text}\nOdia: {or_text}"
    }

  return formatted


def convert_to_jsonl(dataset, output_path, template_format="chatml"):
  with open(output_path, 'w', encoding='utf-8') as f:
    for example in dataset:
      # print(row['eng_string_clean'], row['or_string_clean'])
      formatted=format_for_translation(example, template_format)
      f.write(json.dumps(formatted, ensure_ascii=False)+'\n')
  print(f"Saved {len(dataset)} examples to {output_path}")

In [95]:
import os
def prepare_all_formats(dataset_dict, output_dir="/content/drive/My Drive/formatted_data"):
  os.makedirs(output_dir, exist_ok=True)

  format_options=["chatml", "alpaca", "llama2", "text_only"]

  for split_name, dataset in dataset_dict.items():
    for fmt in format_options:
      output_path=f"{output_dir}/{split_name}_{fmt}.jsonl"
      convert_to_jsonl(dataset, output_path, template_format=fmt)
  print(f"All dataset formats saved to {output_dir}")


In [96]:
prepare_all_formats(hf_dataset)

Saved 73256 examples to /content/drive/My Drive/formatted_data/train_chatml.jsonl
Saved 73256 examples to /content/drive/My Drive/formatted_data/train_alpaca.jsonl
Saved 73256 examples to /content/drive/My Drive/formatted_data/train_llama2.jsonl
Saved 73256 examples to /content/drive/My Drive/formatted_data/train_text_only.jsonl
Saved 8140 examples to /content/drive/My Drive/formatted_data/validation_chatml.jsonl
Saved 8140 examples to /content/drive/My Drive/formatted_data/validation_alpaca.jsonl
Saved 8140 examples to /content/drive/My Drive/formatted_data/validation_llama2.jsonl
Saved 8140 examples to /content/drive/My Drive/formatted_data/validation_text_only.jsonl
Saved 9044 examples to /content/drive/My Drive/formatted_data/test_chatml.jsonl
Saved 9044 examples to /content/drive/My Drive/formatted_data/test_alpaca.jsonl
Saved 9044 examples to /content/drive/My Drive/formatted_data/test_llama2.jsonl
Saved 9044 examples to /content/drive/My Drive/formatted_data/test_text_only.jsonl

In [97]:
# Preprocessing the instructions
def preprocess_function(examples, max_length=512):
  # tokenize inputs and targets
  if "text" in examples:
    texts=examples["text"]
    model_inputs=base_mistral_tokenizer(texts, max_length=max_length, truncation=True)
    return model_inputs

  else:
    model_inputs={}
    for i in range(len(examples["instruction"])):
      text = f"### Instruction: {examples['instruction'][i]}\n"
      text += f"### Input: {examples['input'][i]}\n"
      text += f"### Response: {examples['output'][i]}"

      tokenized=base_mistral_tokenizer(text, max_length=max_length, truncation=True)

      # Add to batch
      for k, v in tokenized.items():
        if k not in model_inputs:
          model_inputs[k]=[]
        model_inputs[k].append(v)
    return model_inputs


In [ ]:
# mistral SFT trainer
from transformers import (
    AutoTokenizer, AutoModelForCausalLM,
    Seq2SeqTrainer, Seq2SeqTrainingArguments,
    DataCollatorForSeq2Seq
)
from datasets import DatasetDict
import torch
import os

class MistralSFTTrainer:
    def __init__(self, model_name, tokenizer_wrapper, dataset: DatasetDict, output_dir="/content/drive/My Drive/mistral-sft-odia"):
        self.model_name = model_name
        self.tokenizer = tokenizer_wrapper.tokenizer  # Base tokenizer
        self.odia_tokenizer = tokenizer_wrapper       # Preprocessing wrapper
        self.dataset = dataset
        self.output_dir = output_dir

        # Load model
        print("📦 Loading model...")
        self.model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.float16)

        # Training arguments
        self.training_args = Seq2SeqTrainingArguments(
            output_dir=self.output_dir,
            evaluation_strategy="steps",
            eval_steps=500,
            save_steps=500,
            logging_dir=os.path.join(self.output_dir, "logs"),
            logging_strategy="steps",
            logging_steps=100,
            save_total_limit=2,
            per_device_train_batch_size=2,
            per_device_eval_batch_size=2,
            num_train_epochs=3,
            predict_with_generate=True,
            fp16=True,
            report_to=["tensorboard"],
            load_best_model_at_end=True,
        )

        # Data collator
        self.data_collator = DataCollatorForSeq2Seq(
            tokenizer=self.tokenizer,
            model=self.model,
            padding=True
        )

    def tokenize_function(self, example):
        src = example["eng_string"]
        tgt = example["or_string"]

        # Encode source
        source_enc = self.tokenizer(
            src,
            truncation=True,
            max_length=512,

        )

        # Preprocess + encode target
        tgt_processed = self.odia_tokenizer.preprocess_text(tgt)
        target_enc = self.tokenizer(
            tgt_processed,
            truncation=True,
            max_length=512,
        )

        return {
            "input_ids": source_enc["input_ids"],
            "attention_mask": source_enc["attention_mask"],
            "labels": target_enc["input_ids"]
        }

    def prepare_data(self):
        print("🧪 Tokenizing dataset...")
        tokenized = self.dataset.map(self.tokenize_function, batched=False)
        self.tokenized_dataset = tokenized

    def train(self):
        print("🚀 Starting training...")
        trainer = Seq2SeqTrainer(
            model=self.model,
            args=self.training_args,
            train_dataset=self.tokenized_dataset["train"],
            eval_dataset=self.tokenized_dataset["validation"],
            tokenizer=self.tokenizer,
            data_collator=self.data_collator
        )
        trainer.train()
        trainer.save_model()
        self.trainer = trainer  # Store for later use

    def evaluate(self, split="test"):
        print(f"📊 Evaluating on {split} set...")
        results = self.trainer.evaluate(self.tokenized_dataset[split])
        print(results)
        return results

    def push_to_hub(self, repo_name):
        print(f"📤 Pushing to hub: {repo_name}")
        self.model.push_to_hub(repo_name)
        self.tokenizer.push_to_hub(repo_name)


In [ ]:
from transformers import AutoTokenizer


trainer = MistralSFTTrainer(
    model_name="mistralai/Mistral-7B-Instruct-v0.3",
    tokenizer_wrapper=odia_tokenizer,
    dataset=dataset,
    output_dir="/content/drive/My Drive/mistral-sft-odia"
)

# 5. Prepare data (tokenization)
trainer.prepare_data()

# 7. Evaluate
# trainer.evaluate("test")

📦 Loading model...


config.json:   0%|          | 0.00/601 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

Fetching 3 files:   0%|          | 0/3 [00:00<?, ?it/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

...

Streaming output truncated to the last 5000 lines.
✅ Final stem: 'ଲସ୍'
✅ Final stem: 'ଏଞ୍ଜେଲସ୍'
✅ Final stem: 'ଟାଇମ୍ସର'
✅ Final stem: 'ମାର୍ଟିନ୍'
✅ Final stem: 'ସାଇ'
✅ Final stem: 'ଲେଖିଥିଲେ'
✅ Final stem: 'ଯେ'
✅ Final stem: '"ସତର୍କମୂଳକ'
✅ Final stem: 'କାହାଣୀ'
📍 Split (grapheme-safe): 'ଏହାଠାରୁ' → 'ଏହାଠା' + 'ରୁ'
  ✅ Final stem: 'ଏହାଠା'
✅ Final stem: 'ଅଧିକ'
✅ Final stem: 'ପ୍ରାସଙ୍ଗିକ'
✅ Final stem: 'ହୋଇନପାରେ"।'
✅ Final stem: 'ଏହା'
✅ Final stem: 'ଟିଭି'
✅ Final stem: 'ଟୋକିଓ,'
✅ Final stem: 'ଜେବେକ୍,'
✅ Final stem: 'ଓ'
✅ Final stem: 'ୟୋମିକୋ'
✅ Final stem: 'ଆଡଭର୍ଟାଇଜିଂ,'
✅ Final stem: 'ଇଙ୍କ୍.'
✅ Final stem: 'ଦ୍ୱାରା'
✅ Final stem: 'ପ୍ରଯୋଜିତ'
✅ Final stem: 'ହୋଇଥିଲା।'
✅ Final stem: 'ଏହା'
✅ Final stem: 'ତଳେ,'
✅ Final stem: '୩୦୦୦'
✅ Final stem: 'ଓ'
✅ Final stem: '୫୦୦୦'
✅ Final stem: 'ମିଟର'
✅ Final stem: '(୯,୮୦୦'
✅ Final stem: 'ଫୁଟ'
✅ Final stem: 'ଏବଂ'
✅ Final stem: '୧୬,୪୦୦'
✅ Final stem: 'ଫୁଟ)'
📍 Split (grapheme-safe): 'ମଧ୍ୟରେ' → 'ମଧ୍ୟ' + 'ରେ'
  ✅ Final stem: 'ମଧ୍ୟ'
✅ Final stem: 'ପଶ୍ଚିମ'
✅ Final ste

Map:   0%|          | 0/8140 [00:00<?, ? examples/s]

Streaming output truncated to the last 5000 lines.
✅ Final stem: 'ସେତେବେଳେ'
✅ Final stem: 'ହୋଇଥାଏ,'
✅ Final stem: 'ଯେତେବେଳେ'
✅ Final stem: 'ଜଣେ'
✅ Final stem: 'ବ୍ୟକ୍ତି'
✅ Final stem: 'ଯକ୍ଷ୍ମାର'
✅ Final stem: 'ଏକ'
✅ Final stem: 'ପ୍ରତିରୋଧକ'
✅ Final stem: 'ଜୀବାଣୁ'
✅ Final stem: 'ଦ୍ୱାରା'
✅ Final stem: 'ସଂକ୍ରମିତ'
✅ Final stem: 'ହୁଅନ୍ତି।'
✅ Final stem: 'ଦୁର୍ଘଟଣାର'
✅ Final stem: 'କିଛି'
✅ Final stem: 'ଦିନ'
✅ Final stem: 'ପରେ'
📍 Split (grapheme-safe): 'ତାଙ୍କୁ' → 'ତା' + 'ଙ୍କୁ'
  ✅ Final stem: 'ତା'
✅ Final stem: 'କାଚୋଇରା'
✅ Final stem: 'ଦୋ'
✅ Final stem: 'ସୁଲ୍ରେ'
✅ Final stem: 'ସମାଧି'
✅ Final stem: 'ଦିଆଯାଇଥିଲା'
✅ Final stem: '।'
📍 Split (grapheme-safe): 'ଗିବନ୍ସଙ୍କ' → 'ଗିବନ୍ସ' + 'ଙ୍କ'
  ✅ Final stem: 'ଗିବନ୍ସ'
✅ Final stem: 'ଗିରଫଦାରୀ'
✅ Final stem: 'ଦ୍ୱାରା'
✅ Final stem: 'ଆନ୍ତର୍ଜାତୀୟ'
📍 Split (grapheme-safe): 'ସ୍ତରରେ' → 'ସ୍ତର' + 'ରେ'
  ✅ Final stem: 'ସ୍ତର'
✅ Final stem: 'ଦୃଢ଼'
✅ Final stem: 'ପ୍ରତିବାଦ'
✅ Final stem: 'ହୋଇଥିଲା'
✅ Final stem: 'ଏବଂ'
✅ Final stem: 'ସୁଦାନ'
✅ Final stem: 'ଏବଂ'
✅ Final stem

Map:   0%|          | 0/9044 [00:00<?, ? examples/s]

Streaming output truncated to the last 5000 lines.
✅ Final stem: 'କରାଏ।'
✅ Final stem: 'ଶୀତଦିନେ'
✅ Final stem: 'ଜାଁସ୍କର'
✅ Final stem: 'ବରଫ'
✅ Final stem: 'ପାଲଟିଯାଏ'
✅ Final stem: 'ଏବଂ'
✅ Final stem: 'ଏହି'
✅ Final stem: 'ଚମତ୍କାର'
✅ Final stem: 'ବରଫାବୃତ'
📍 Split (grapheme-safe): 'ନଦୀରେ' → 'ନଦୀ' + 'ରେ'
  ✅ Final stem: 'ନଦୀ'
✅ Final stem: 'ପ୍ରସିଦ୍ଧ'
✅ Final stem: 'ଚାଦର'
✅ Final stem: 'ଟ୍ରେକ୍'
✅ Final stem: 'ଆୟୋଜିତ'
✅ Final stem: 'ହୋଇଥାଏ।'
📍 Split (grapheme-safe): 'ମିନାଣ୍ଡରଙ୍କ' → 'ମିନାଣ୍ଡର' + 'ଙ୍କ'
  ✅ Final stem: 'ମିନାଣ୍ଡର'
✅ Final stem: 'ମୃତ୍ୟୁ'
✅ Final stem: 'ପରେ'
✅ Final stem: 'ପରେ,'
📍 Split (grapheme-safe): 'ତାଙ୍କ' → 'ତା' + 'ଙ୍କ'
  ✅ Final stem: 'ତା'
✅ Final stem: 'ସାମ୍ରାଜ୍ୟର'
✅ Final stem: 'ଅଧିକାଂଶ'
✅ Final stem: 'ଭାଗ'
✅ Final stem: 'ବିଖଣ୍ଡିତ'
✅ Final stem: 'ହୋଇଯାଇଥିଲା'
✅ Final stem: 'ଏବଂ'
🔨 Hyphen split: 'ଇଣ୍ଡୋ-ଗ୍ରୀକ୍' → 'ଇଣ୍ଡୋ ଗ୍ରୀକ୍'
✅ Final stem: 'ଇଣ୍ଡୋ ଗ୍ରୀକ୍'
✅ Final stem: 'ପ୍ରଭାବ'
✅ Final stem: 'ଯଥେଷ୍ଟ'
✅ Final stem: 'ହ୍ରାସ'
✅ Final stem: 'ପାଇଥିଲା।'
✅ Final stem: 'ସ୍ପର୍ଶର'
✅ F

In [ ]:
trainer.train()

🚀 Starting training...


<ipython-input-13-912dbc024fdc>:82: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


ValueError: Asking to pad but the tokenizer does not have a padding token. Please select a token to use as `pad_token` `(tokenizer.pad_token = tokenizer.eos_token e.g.)` or add a new pad token via `tokenizer.add_special_tokens({'pad_token': '[PAD]'})`.

In [ ]:
%load_ext tensorboard
%tensorboard --logdir /content/drive/My Drive/mistral-sft-odia/logs


The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


ERROR: Failed to launch TensorBoard (exited with 2).
Contents of stderr:
2025-03-31 16:44:44.508565: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1743439484.527098    9528 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1743439484.532410    9528 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
usage: tensorboard [-h] [--helpfull] [--logdir PATH] [--logdir_spec PATH_SPEC] [--host ADDR]
                   [--bind_all] [--port PORT] [--reuse_port BOOL] [--load_fast {false,auto,true}]
                   [--extra_data_server_flags EXTRA_DATA_SERVER_FLAGS]
                   [--grpc_creds_type {local,ssl,ssl_dev}] [--grpc_data_provider PORT]
               